In [1]:
import torch.nn as nn
import math, pdb
import torch.nn.functional as F
import torch
from torch.nn.parameter import Parameter
from nonlocal_helper import Nonlocal
from blocks import *
from backbones import swin_transformer
import modules.utils as lutils

import yaml
from easydict import EasyDict
from backbones import AVA_backbone
from einops import rearrange


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint
import numpy as np
from timm.models.layers import DropPath, trunc_normal_

from mmengine.runner import load_checkpoint
# from mmaction.utils import get_root_logger
from builder import BACKBONES

from mmengine.logging import MMLogger


from functools import reduce, lru_cache
from operator import mul
from einops import rearrange

#good

In [2]:
#good
### Download weights from https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
def conv3x3(in_planes, out_planes, stride=1, padding=1, bias=False):
    """3x3 convolution with padding"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=(1, 3, 3), stride=(1, stride, stride),
                     padding=(0,padding,padding), bias=bias)


def conv1x1(in_channel, out_channel):
    return nn.Conv3d(in_channel, out_channel, kernel_size=1, stride=1, padding=0, bias=False)

In [3]:
#good


class ResNetFPN(nn.Module):

    def __init__(self, block, args, pretrained=None,
                 pretrained2d=True,
                 patch_size=(2, 4, 4),
        embed_dim=192,
        depths=[2, 2, 18, 2],
        num_heads=[6, 12, 24, 48],
        window_size=(8, 7, 7),
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
                 drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.4,
        patch_norm=True,
                 norm_layer=nn.LayerNorm,
                 frozen_stages=-1,
                 use_checkpoint=False):
        
        self.inplanes = 64
        super(ResNetFPN, self).__init__()
    
    
        if args.MODEL_TYPE.startswith('SlowFast'):
            with open('configs/ROAD.yml') as f:
                config = yaml.load(f, Loader=yaml.FullLoader)
            opt = EasyDict(config)
            print(opt)
            self.inplanes = 64
            super(ResNetFPN, self).__init__()
            self.backbone = AVA_backbone(opt)
    


        depths=[2, 2, 18, 2]
        num_heads=[6, 12, 24, 48]
        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))]  # stochastic depth decay rule


        self.pretrained = pretrained
        self.pretrained2d = pretrained2d
        self.num_layers = len(depths)
        self.embed_dim = embed_dim
        self.patch_norm = patch_norm
        self.frozen_stages = frozen_stages
        self.window_size = window_size
        self.patch_size = patch_size



        self.MODEL_TYPE = args.MODEL_TYPE
        num_blocks = args.model_perms
        non_local_inds = args.non_local_inds
        model_3d_layers = args.model_3d_layers
        self.num_blocks = num_blocks
        self.non_local_inds = non_local_inds
        self.model_3d_layers = model_3d_layers
        self.patch_norm = True
        self.patch_size=patch_size
        self.window_size=window_size
        self.num_layers=4
        self.num_features = int(embed_dim * 2**(self.num_layers-1))
        self.patch_embed=swin_transformer.PatchEmbed3D(patch_size=self.patch_size, in_chans=3, embed_dim=embed_dim,
            norm_layer=nn.LayerNorm if norm_layer else None)#embed_dim 4 56 56
        
        self.pos_drop = nn.Dropout(p=0.0)
        # self.layers = nn.ModuleList()
        
        self.swinaclayer1=swin_transformer.BasicLayer(dim=int(embed_dim*2**0),depth=depths[0],num_heads=num_heads[0],window_size=self.window_size,
                                                      qkv_bias=True,drop_path=dpr[sum(depths[:0]):sum(depths[:1])],downsample=swin_transformer.PatchMerging if 0<self.num_layers-1 else None)#384 4 28 28
        
        # self.patchmerge1=swin_transformer.PatchMerging(dim=192)#384 4 28 28
        
        self.swinaclayer2=swin_transformer.BasicLayer(dim=embed_dim*2**1,depth=2,num_heads=12,window_size=(8,7,7),
                                                      qkv_bias=True,drop_path=dpr[sum(depths[:1]):sum(depths[:2])],downsample=swin_transformer.PatchMerging if 1<self.num_layers-1 else None)
        
        # self.patchmerge2=swin_transformer.PatchMerging(dim=192*2)#768 4 14 14
        
        self.swinaclayer3=swin_transformer.BasicLayer(dim=embed_dim*2**2,depth=18,num_heads=24,window_size=(8,7,7),
                                                      qkv_bias=True,drop_path=dpr[sum(depths[:2]):sum(depths[:3])],downsample=swin_transformer.PatchMerging if 2<self.num_layers-1 else None)#768 4 14 14
        
        # self.patchmerge3=swin_transformer.PatchMerging(dim=192*2**2)#1536 4 7 7
        
        self.swinaclayer4=swin_transformer.BasicLayer(dim=embed_dim*2**3,depth=2,num_heads=48,window_size=(8,7,7),
                                                      qkv_bias=True,drop_path=dpr[sum(depths[:3]):sum(depths[:3 + 1])],downsample=swin_transformer.PatchMerging if 3<self.num_layers-1 else None)#1536 4 7 7
        # self.layers.append(layer)
        

        





        self.norm3 = nn.LayerNorm(self.num_features)

        self._freeze_stages()
        
        
        self.upcov1=nn.Conv3d(384, 256, kernel_size=1, stride=1, padding=0, bias=False)
        self.upcov2=nn.ConvTranspose3d(768, 512, kernel_size=(1,2,2), stride=(1,2,2))
        self.upcov3=nn.ConvTranspose3d(1536, 1024, kernel_size=(1,2,2), stride=(1,2,2))
        self.upcov4=nn.Conv3d(1536, 2048, kernel_size=1, stride=1, padding=0, bias=False)
        # self.swinlayer1=swinmodel.layers[0]
        # self.swinlayer2=swinmodel.layers[1]
        # self.swinlayer3=swinmodel.layers[2]
        # self.swinlayer4=swinmodel.layers[3]

        

        # self.conv1 = nn.Conv3d(3, 64, kernel_size=(1,7,7), stride=(1,2,2), padding=(0,3,3),
        #                        bias=False)
        # self.bn1 = nn.BatchNorm3d(64)
        # self.relu = nn.ReLU(inplace=True)
        # self.pool1 = nn.MaxPool3d(kernel_size=(
        #     1, 3, 3), stride=(1, 2, 2), padding=(0, 0, 0))
        self.layer_names = []

        self.layer1 = self._make_layer(
            block, 64, num_blocks[0], temp_kernals=model_3d_layers[0], nl_inds=non_local_inds[0])
        self.MODEL_TYPE = args.MODEL_TYPE
        
        # if args.model_subtype in ['C2D','RCN','CLSTM','RCLSTM','CGRU','RCGRU']:
        #     self.pool2 = None
        # else:
        #     self.pool2 = nn.MaxPool3d(kernel_size=(
        #         2, 1, 1), stride=(2, 1, 1), padding=(0, 0, 0))
        self.pool2=None
        self.layer2 = self._make_layer(
            block, 128, num_blocks[1], stride=2, temp_kernals=model_3d_layers[1], nl_inds=non_local_inds[1])
        self.layer3 = self._make_layer(
            block, 256, num_blocks[2], stride=2, temp_kernals=model_3d_layers[2], nl_inds=non_local_inds[2])
        self.layer4 = self._make_layer(
            block, 512, num_blocks[3], stride=2, temp_kernals=model_3d_layers[3], nl_inds=non_local_inds[3])

        #self.avgpool = nn.AvgPool2d(7, stride=1)
        #self.fc = nn.Linear(512 * block.expansion, num_classes)

        if self.MODEL_TYPE == 'SlowFast':
            self.conv6 = conv3x3(2304, 256, stride=2, padding=1)  # P6
            self.conv7 = conv3x3(256, 256, stride=2, padding=1)  # P7

            self.ego_lateral = conv3x3(512 * block.expansion,  256, stride=2, padding=0)
            self.avg_pool = nn.AdaptiveAvgPool3d((None, 1, 1))

            self.lateral_layer1 = conv1x1(2304, 256)
            self.lateral_layer2 = conv1x1(1152, 256)
            self.lateral_layer3 = conv1x1(576, 256)
            
            self.corr_layer1 = conv3x3(256, 256, stride=1, padding=1)  # P4
            self.corr_layer2 = conv3x3(256, 256, stride=1, padding=1)  # P4
            self.corr_layer3 = conv3x3(256, 256, stride=1, padding=1)  # P3
        else:
            self.conv6 = conv3x3(512 * block.expansion, 256, stride=2, padding=1)  # P6
            self.conv7 = conv3x3(256, 256, stride=2, padding=1)  # P7

            # self.ego_lateral = conv3x3(512 * block.expansion,  256, stride=2, padding=0)
            self.avg_pool = nn.AdaptiveAvgPool3d((None, 1, 1))

            self.lateral_layer1 = conv1x1(512 * block.expansion, 256)
            self.lateral_layer2 = conv1x1(256 * block.expansion, 256)
            self.lateral_layer3 = conv1x1(128 * block.expansion, 256)
            
            self.corr_layer1 = conv3x3(256, 256, stride=1, padding=1)  # P4
            self.corr_layer2 = conv3x3(256, 256, stride=1, padding=1)  # P4
            self.corr_layer3 = conv3x3(256, 256, stride=1, padding=1)  # P3


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_uniform_(m.weight, a=1)
                if hasattr(m.bias, 'data'):
                    torch.nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


    

    def _freeze_stages(self):
        if self.frozen_stages >= 0:
            self.patch_embed.eval()
            for param in self.patch_embed.parameters():
                param.requires_grad = False

        if self.frozen_stages >= 1:
            self.pos_drop.eval()
            for i in range(0, self.frozen_stages):
                m = self.layers[i]
                m.eval()
                for param in m.parameters():
                    param.requires_grad = False




    def _upsample(self, x, y):
        _, _, t, h, w = y.size()
        # print('spatial', x.shape, y.shape)
        x_upsampled = F.interpolate(x, [t, h, w], mode='nearest')

        return x_upsampled

    def _upsample_time(self, x):
        _,_,t, h, w = x.size()
        # print('time', x.shape)
        x_upsampled = F.interpolate(x, [t*2, h, w], mode='nearest')

        return x_upsampled

    def _make_layer(self, block, planes, num_blocks, stride=1, temp_kernals=[], nl_inds=[]):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv3d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=(1, stride, stride), bias=False),
                nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layer_names = []
        if 0 in temp_kernals:
            layers.append(block(self.inplanes, planes, stride,
                                temp_kernal=3, downsample=downsample))
        else:
            layers.append(block(self.inplanes, planes, stride,
                                temp_kernal=0, downsample=downsample))
        layer_names.append(0)
        self.inplanes = planes * block.expansion

        for i in range(1, num_blocks):
            if i in temp_kernals:
                layers.append(block(self.inplanes, planes, temp_kernal=3))
            else:
                layers.append(block(self.inplanes, planes, temp_kernal=1))
            layer_names.append(i)

            if i in nl_inds:
                nln = Nonlocal(
                    planes * block.expansion,
                    (planes * block.expansion) // 2,
                    [1, 2, 2],
                    instantiation='softmax',
                )
                layers.append(nln)

        self.layer_names.append(layer_names)

        return nn.Sequential(*layers)

    def forward(self, x):

        if self.MODEL_TYPE.startswith('SlowFast'):
            ff = self.backbone(x)
            c3 = ff[0]
            c4 = ff[1]
            c5 = ff[2]
            p5 = self.lateral_layer1(c5)
            p5_upsampled = self._upsample(p5, c4)
            p5 = self.corr_layer1(p5)
            p4 = self.lateral_layer2(c4)
            p4 = p5_upsampled + p4
            p4_upsampled = self._upsample(p4, c3)
            p4 = self.corr_layer2(p4)
            p3 = self.lateral_layer3(c3)
            p3 = p4_upsampled + p3
            p3 = self.corr_layer3(p3)
            p6 = self.conv6(c5)
            p7 = self.conv7(F.relu(p6))
            features = [p3, p4, p5, p6, p7]
            ego_feat = self.avg_pool(p7)
            if self.pool2 is not None:
                for i in range(len(features)):
                    features[i] = self._upsample_time(features[i])
                ego_feat = self._upsample_time(ego_feat)
        else:
            x=self.patch_embed(x)
            # print(x.shape)
            x=self.pos_drop(x)
            # print(x.shape)
            x=self.swinaclayer1(x)
            # print(x.shape)
            x1=x #192 4 56 56
            # x=self.patchmerge1(x)
            x=self.swinaclayer2(x)
            x2=x #384 4 28 28
            # x=self.patchmerge2(x)
            x=self.swinaclayer3(x)
            x3=x #768 4 14 14
            # x=self.patchmerge3(x)
            x=self.swinaclayer4(x)
            x = rearrange(x, 'n c d h w -> n d h w c')
            x = self.norm3(x)
            x = rearrange(x, 'n d h w c -> n c d h w')
            x4=x #1536 4 7 7

            #up can be pretrained

            u1=self.upcov1(x1)#256 4 56 56
            u2=self.upcov2(x2)#512 4 28 28
            u3=self.upcov3(x3)#1024 4 14 14
            u4=self.upcov4(x4)#2048 4 7 7
            
            
            
            
            # x = self.conv1(x)
            # x = self.bn1(x)
            # x = self.relu(x)
            # x = self.pool1(x)
            x = u1
            # if self.pool2 is not None:
            #     x = self.pool2(x)
            c3 = u2
            c4 = u3
            c5 = u4

            #down can be pretrained

            p5 = self.lateral_layer1(c5)
            p5_upsampled = self._upsample(p5, c4)
            p5 = self.corr_layer1(p5)
            p4 = self.lateral_layer2(c4)
            p4 = p5_upsampled + p4
            p4_upsampled = self._upsample(p4, c3)
            p4 = self.corr_layer2(p4)
            p3 = self.lateral_layer3(c3)
            p3 = p4_upsampled + p3
            p3 = self.corr_layer3(p3)
            p6 = self.conv6(c5)
            p7 = self.conv7(F.relu(p6))
            features = [p3, p4, p5, p6, p7]
            # ego_feat = self.avg_pool(p7)
            if self.pool2 is not None:
                for i in range(len(features)):
                    features[i] = self._upsample_time(features[i])
                # ego_feat = self._upsample_time(ego_feat)
        return features



In [4]:
#good


import argparse
import datetime
import random
import sys

import numpy as np
import torch
from torchvision import transforms

# import data.transforms as vtf
# from data import VideoDataset
# from gen_dets import gen_dets
# from retinanet import build_retinanet
# from modules import utils
# from train import train
# from tubes import build_eval_tubes
# from utils_ssl import save_ulb_indices
# from val import val


def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")


def seed_everything(args):
    random.seed(args.MAN_SEED)
    np.random.seed(args.MAN_SEED)
    torch.manual_seed(args.MAN_SEED)
    torch.cuda.manual_seed_all(args.MAN_SEED)



parser = argparse.ArgumentParser(description='Training single stage FPN with OHEM, resnet as backbone')
# parser.add_argument('TASK', type=int, choices=[1,2], help="if task 1, using only 3 out of 15 videos from the training parition; if task 2, using all 15 videos from the training partition train_1")
# parser.add_argument('DATA_ROOT', help='Location to root directory for dataset reading') # /mnt/mars-fast/datasets/
# parser.add_argument('SAVE_ROOT', help='Location to root directory for saving checkpoint models') # /mnt/mars-alpha/
# parser.add_argument('MODEL_PATH',help='Location to root directory where kinetics pretrained models are stored')

parser.add_argument('--MODE', default='train', choices=['train', 'gen_dets', 'eval_frames', 'val'],
                    help='MODE can be train, gen_dets, eval_frames, val; define SUBSETS accordingly, build tubes')
# Name of backbone network, e.g. resnet18, resnet34, resnet50, resnet101 resnet152 are supported
parser.add_argument('--ARCH', default='resnet50', 
                    type=str, help=' base arch')
parser.add_argument('--MODEL_TYPE', default='RCGRU',
                    type=str, help=' base model')
parser.add_argument('--ANCHOR_TYPE', default='RETINA',
                    type=str, help='type of anchors to be used in model')

parser.add_argument('--SEQ_LEN', default=8,
                    type=int, help='NUmber of input frames')
parser.add_argument('--TEST_SEQ_LEN', default=8,
                    type=int, help='NUmber of input frames')
parser.add_argument('--MIN_SEQ_STEP', default=1,
                    type=int, help='DIFFERENCE of gap between the frames of sequence')
parser.add_argument('--MAX_SEQ_STEP', default=1,
                    type=int, help='DIFFERENCE of gap between the frames of sequence')
# if output heads are have shared features or not: 0 is no-shareing else sharining enabled
# parser.add_argument('--MULIT_SCALE', default=False, type=str2bool,help='perfrom multiscale training')
parser.add_argument('--HEAD_LAYERS', default=3, 
                    type=int,help='0 mean no shareding more than 0 means shareing')
parser.add_argument('--NUM_FEATURE_MAPS', default=5, 
                    type=int,help='0 mean no shareding more than 0 means shareing')
parser.add_argument('--CLS_HEAD_TIME_SIZE', default=3, 
                    type=int, help='Temporal kernel size of classification head')
parser.add_argument('--REG_HEAD_TIME_SIZE', default=3,
                type=int, help='Temporal kernel size of regression head')

#  Name of the dataset only voc or coco are supported
parser.add_argument('--DATASET', default='road', 
                    type=str,help='dataset being used')
parser.add_argument('--TRAIN_SUBSETS', default='train_1,',
                    type=str,help='Training SUBSETS seprated by ,')
parser.add_argument('--VAL_SUBSETS', default='val_1',
                    type=str,help='Validation SUBSETS seprated by ,')
parser.add_argument('--TEST_SUBSETS', default='', 
                    type=str,help='Testing SUBSETS seprated by ,')
# Input size of image only 600 is supprted at the moment 
parser.add_argument('--MIN_SIZE', default=512, 
                    type=int, help='Input Size for FPN')

#  data loading argumnets
parser.add_argument('-b','--BATCH_SIZE', default=4, 
                    type=int, help='Batch size for training')
parser.add_argument('--TEST_BATCH_SIZE', default=1, 
                    type=int, help='Batch size for testing')
# Number of worker to load data in parllel
parser.add_argument('--NUM_WORKERS', '-j', default=8, 
                    type=int, help='Number of workers used in dataloading')
# optimiser hyperparameters
parser.add_argument('--OPTIM', default='SGD', 
                    type=str, help='Optimiser type')
parser.add_argument('--RESUME', default=0, 
                    type=int, help='Resume from given epoch')
parser.add_argument('--MAX_EPOCHS', default=150,
                    type=int, help='Number of training epoc')
parser.add_argument('-l','--LR', '--learning-rate', 
                    default=0.0041, type=float, help='initial learning rate')
parser.add_argument('--MOMENTUM', default=0.9, 
                    type=float, help='momentum')
parser.add_argument('--MILESTONES', default='130,145',
                    type=str, help='Chnage the lr @')
parser.add_argument('--GAMMA', default=0.1, 
                    type=float, help='Gamma update for SGD')
parser.add_argument('--WEIGHT_DECAY', default=1e-4, 
                    type=float, help='Weight decay for SGD')

# Freeze layers or not 
parser.add_argument('--FBN','--FREEZE_BN', default=True, 
                    type=str2bool, help='freeze bn layers if true or else keep updating bn layers')
parser.add_argument('--FREEZE_UPTO', default=1, 
                    type=int, help='layer group number in ResNet up to which needs to be frozen')

# Loss function matching threshold
parser.add_argument('--POSTIVE_THRESHOLD', default=0.5, 
                    type=float, help='Min threshold for Jaccard index for matching')
parser.add_argument('--NEGTIVE_THRESHOLD', default=0.4,
                    type=float, help='Max threshold Jaccard index for matching')
# Evaluation hyperparameters
parser.add_argument('--EVAL_EPOCHS', default='150',
                    type=str, help='eval epochs to test network on these epoch checkpoints usually the last epoch is used')
parser.add_argument('--VAL_STEP', default=10,
                    type=int, help='Number of training epoch before evaluation')
parser.add_argument('--IOU_THRESH', default=0.5, 
                    type=float, help='Evaluation threshold for validation and for frame-wise mAP')
parser.add_argument('--CONF_THRESH', default=0.025, 
                    type=float, help='Confidence threshold for to remove detection below given number')
parser.add_argument('--NMS_THRESH', default=0.5, 
                    type=float, help='NMS threshold to apply nms at the time of validation')
parser.add_argument('--TOPK', default=10, 
                    type=int, help='topk detection to keep for evaluation')
parser.add_argument('--GEN_CONF_THRESH', default=0.025, 
                    type=float, help='Confidence threshold at the time of generation and dumping')
parser.add_argument('--GEN_TOPK', default=100, 
                    type=int, help='topk at the time of generation')
parser.add_argument('--GEN_NMS', default=0.5, 
                    type=float, help='NMS at the time of generation')
parser.add_argument('--CLASSWISE_NMS', default=False, 
                    type=str2bool, help='apply classwise NMS/no tested properly')
parser.add_argument('--JOINT_4M_MARGINALS', default=False, 
                    type=str2bool, help='generate score of joints i.e. duplexes or triplet by marginals like agents and actions scores')

## paths hyper parameters
parser.add_argument('--COMPUTE_PATHS', default=False, 
                    type=str2bool, help=' COMPUTE_PATHS if set true then it overwrite existing ones')
parser.add_argument('--PATHS_IOUTH', default=0.5,
                    type=float, help='Iou threshold for building paths to limit neighborhood search')
parser.add_argument('--PATHS_COST_TYPE', default='score',
                    type=str, help='cost function type to use for matching, other options are scoreiou, iou')
parser.add_argument('--PATHS_JUMP_GAP', default=4,
                    type=int, help='GAP allowed for a tube to be kept alive after no matching detection found')
parser.add_argument('--PATHS_MIN_LEN', default=6,
                    type=int, help='minimum length of generated path')
parser.add_argument('--PATHS_MINSCORE', default=0.1,
                    type=float, help='minimum score a path should have over its length')

## paths hyper parameters
parser.add_argument('--COMPUTE_TUBES', default=False, type=str2bool, help='if set true then it overwrite existing tubes')
parser.add_argument('--TUBES_ALPHA', default=0,
                    type=float, help='alpha cost for changeing the label')
parser.add_argument('--TRIM_METHOD', default='none',
                    type=str, help='other one is indiv which works for UCF24')
parser.add_argument('--TUBES_TOPK', default=10,
                    type=int, help='Number of labels to assign for a tube')
parser.add_argument('--TUBES_MINLEN', default=5,
                    type=int, help='minimum length of a tube')
parser.add_argument('--TUBES_EVAL_THRESHS', default='0.2,0.5',
                    type=str, help='evaluation threshold for checking tube overlap at evaluation time, one can provide as many as one wants')
# parser.add_argument('--TRAIL_ID', default=0,
#                     type=int, help='eval TUBES_Thtrshold at evaluation time')

###
parser.add_argument('--LOG_START', default=10, 
                    type=int, help='start loging after k steps for text/tensorboard') 
parser.add_argument('--LOG_STEP', default=10, 
                    type=int, help='Log every k steps for text/tensorboard')
parser.add_argument('--TENSORBOARD', default=1,
                    type=str2bool, help='Use tensorboard for loss/evalaution visualization')

# Program arguments
parser.add_argument('--MAN_SEED', default=123, 
                    type=int, help='manualseed for reproduction')
parser.add_argument('--MULTI_GPUS', default=True, type=str2bool, help='If  more than 0 then use all visible GPUs by default only one GPU used ') 

parser.add_argument('--LOGIC', default='None', type=str, help='T-norm to be used in the loss')
parser.add_argument('--req_loss_weight', default=0.0, type=float, help='weight for the logic-based loss')
parser.add_argument('--DEBUG_num_iter', default=0, type=int, help='num iterations to run; fast debugging')
parser.add_argument('--EXP_NAME', default="", type=str, help="Custom experiment name, for resuming and eval, provide the full path to the experiment directory")

parser.add_argument('--pretrained_model_path', default=None, type=str, help='path to pretrained model; e.g. use this to add warm-up to neurosymbolic training with tnorm based loss')
parser.add_argument('--unlabelled_proportion', default=0.0, type=float)
parser.add_argument('--agentness_th', default=0.125, type=float, help='threshold to distinguish foreground vs background boxes when computing t-norm')
parser.add_argument('--model_perms',default=[3, 4, 6, 3],type=list,help='model perms')
parser.add_argument('--model_3d_layers',default=[[0, 1, 2], [0, 2], [0, 2, 4], [0, 1]],type=list,help='model 3d layers')
parser.add_argument('--non_local_inds',default=[[], [], [], []],type=list,help='non local inds')
parser.add_argument('--num_classes',default=25,type=int,help='num class')
parser.add_argument('--num_classes_list',default=[1, 24],type=list,help='class list')
parser.add_argument('--label_types',default=['action_ness', 'action'],type=list,help='label types')
parser.add_argument('--num_label_types',default=2,type=int,help='num label types')
train_dataset = None
ulb_train_dataset = None

# args.model_perms = [3, 4, 6, 3]
# args.model_3d_layers = [[0, 1, 2], [0, 2], [0, 2, 4], [0, 1]]
# args.non_local_inds=[[], [], [], []]
# assert args.ARCH in modelperms, 'Arch shoudl from::>' + \
#     ','.join([m for m in modelperms])

## Parse arguments
args, unknown = parser.parse_known_args()

import torchinfo
from torchinfo import summary


args.model_perms = [3, 4, 6, 3]
args.num_classes=25
args.model_3d_layers = [[0, 1, 2], [0, 2], [0, 2, 4], [0, 1]]
args.non_local_inds=[[], [], [], []]



In [9]:
import collections


def exract():
# 查看backbone部分的预训练模型键值对
    backbone_model_path = 'F:\\gta5\\swin-large-p244-w877_in22k-pre_16xb8-amp-32x2x1-30e_kinetics700-rgb_20220930-f8d74db7.pth'
    backbone_train_model = torch.load(backbone_model_path)
    state_dict = collections.OrderedDict(backbone_train_model['state_dict'])
    print(list(state_dict.keys()))


In [10]:
exract()

['backbone.patch_embed.proj.weight', 'backbone.patch_embed.proj.bias', 'backbone.patch_embed.norm.weight', 'backbone.patch_embed.norm.bias', 'backbone.layers.0.blocks.0.norm1.weight', 'backbone.layers.0.blocks.0.norm1.bias', 'backbone.layers.0.blocks.0.attn.relative_position_bias_table', 'backbone.layers.0.blocks.0.attn.relative_position_index', 'backbone.layers.0.blocks.0.attn.qkv.weight', 'backbone.layers.0.blocks.0.attn.qkv.bias', 'backbone.layers.0.blocks.0.attn.proj.weight', 'backbone.layers.0.blocks.0.attn.proj.bias', 'backbone.layers.0.blocks.0.norm2.weight', 'backbone.layers.0.blocks.0.norm2.bias', 'backbone.layers.0.blocks.0.mlp.fc1.weight', 'backbone.layers.0.blocks.0.mlp.fc1.bias', 'backbone.layers.0.blocks.0.mlp.fc2.weight', 'backbone.layers.0.blocks.0.mlp.fc2.bias', 'backbone.layers.0.blocks.1.norm1.weight', 'backbone.layers.0.blocks.1.norm1.bias', 'backbone.layers.0.blocks.1.attn.relative_position_bias_table', 'backbone.layers.0.blocks.1.attn.relative_position_index', 'ba

In [11]:
model=ResNetFPN(BottleneckRCGRU,args)

model_stat_dict = model.state_dict()
print(model_stat_dict.keys())

d:\Users\belongTH\anaconda3\envs\testpyt\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


odict_keys(['patch_embed.proj.weight', 'patch_embed.proj.bias', 'patch_embed.norm.weight', 'patch_embed.norm.bias', 'swinaclayer1.blocks.0.norm1.weight', 'swinaclayer1.blocks.0.norm1.bias', 'swinaclayer1.blocks.0.attn.relative_position_bias_table', 'swinaclayer1.blocks.0.attn.relative_position_index', 'swinaclayer1.blocks.0.attn.qkv.weight', 'swinaclayer1.blocks.0.attn.qkv.bias', 'swinaclayer1.blocks.0.attn.proj.weight', 'swinaclayer1.blocks.0.attn.proj.bias', 'swinaclayer1.blocks.0.norm2.weight', 'swinaclayer1.blocks.0.norm2.bias', 'swinaclayer1.blocks.0.mlp.fc1.weight', 'swinaclayer1.blocks.0.mlp.fc1.bias', 'swinaclayer1.blocks.0.mlp.fc2.weight', 'swinaclayer1.blocks.0.mlp.fc2.bias', 'swinaclayer1.blocks.1.norm1.weight', 'swinaclayer1.blocks.1.norm1.bias', 'swinaclayer1.blocks.1.attn.relative_position_bias_table', 'swinaclayer1.blocks.1.attn.relative_position_index', 'swinaclayer1.blocks.1.attn.qkv.weight', 'swinaclayer1.blocks.1.attn.qkv.bias', 'swinaclayer1.blocks.1.attn.proj.weigh

In [22]:

#good

# model.eval()
# #只保留权重
# torch.save(model.state_dict(), 'model_no_arc.pth')


checkpoint = torch.load('F:/gta5/swin-large-p244-w877_in22k-pre_16xb8-amp-32x2x1-30e_kinetics700-rgb_20220930-f8d74db7.pth')
state_dict = collections.OrderedDict(checkpoint['state_dict'])
checkpoint2=torch.load('F:/A-ROAD-CHALL/weights/weights_task1/pretrained_weights_task1.pth')
# # # # model.load_state_dict(checkpoint, strict=False)

# # # # load_checkpoint(model, checkpoint, strict=False,revise_keys=[r'^backbone\.'])
backbone_stat_dict = {}
for k in list(state_dict.keys()):
    # 去掉backbone.
    # new_k=k
    if k.startswith('backbone'):
         backbone_stat_dict[k.replace('backbone.', '')] = state_dict[k]
    # 修改层名
    if k.startswith('backbone.layers.0'):    
        backbone_stat_dict[k.replace('backbone.layers.0', 'swinaclayer1')] = state_dict[k]

    if k.startswith('backbone.layers.1'):
        backbone_stat_dict[k.replace('backbone.layers.1', 'swinaclayer2')] = state_dict[k]
    
    if k.startswith('backbone.layers.2'):
        backbone_stat_dict[k.replace('backbone.layers.2', 'swinaclayer3')] = state_dict[k]
    
    if k.startswith('backbone.layers.3'):
        backbone_stat_dict[k.replace('backbone.layers.3', 'swinaclayer4')] = state_dict[k]
    # new_k = new_k.replace('layers.1', 'swinaclayer2')
    # new_k = new_k.replace('layers.2', 'swinaclayer3')
    # # print('ok')
    # new_k = new_k.replace('layers.3', 'swinaclayer4')
    # # 改键名
    # checkpoint[new_k] = state_dict.pop(k)
# model_stat_dict.update(backbone_stat_dict)
# model.load_state_dict(model_stat_dict,strict=False)
# backbone_stat_dict.update(state_dict)

backbone_stat_dict2 = {}
for k in list(checkpoint2.keys()):
    if 'module.backbone.layer' or 'conv6' or 'conv7' or 'lateral' or 'corr_layer' or 'heads' in k:
        if k.startswith('module.backbone'):
            backbone_stat_dict2[k.replace('module.backbone.', '')] = checkpoint2[k]

        # if k.startswith('module'):
        #     backbone_stat_dict2[k.replace('module.', '')] = checkpoint2[k]



new_stat_dict={}
new_stat_dict.update(backbone_stat_dict2)
new_stat_dict.update(backbone_stat_dict)
# for k in list(checkpoint2.keys()):
#     new_k=k
#     if k.startswith('module.backbone'):
#         new_k = k[16:]
#         # print('ok')
#     #改键名
#     checkpoint2[new_k] = checkpoint2.pop(k)
model.load_state_dict(new_stat_dict,strict=False)

# new_checkpoint = {}
# new_checkpoint.update(checkpoint) 
# new_checkpoint.update(checkpoint2)

# 加载到模型中    
# model.load_state_dict(new_checkpoint, strict=False)
# model.load_state_dict(checkpoint2, strict=False)
# model.load_state_dict(state_dict, strict=False)


# summary(model, input_size=(1, 3, 8, 224, 224))
# print(model)


_IncompatibleKeys(missing_keys=['upcov1.weight', 'upcov2.weight', 'upcov2.bias', 'upcov3.weight', 'upcov3.bias', 'upcov4.weight'], unexpected_keys=['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layers.0.blocks.0.norm1.weight', 'layers.0.blocks.0.norm1.bias', 'layers.0.blocks.0.attn.relative_position_bias_table', 'layers.0.blocks.0.attn.relative_position_index', 'layers.0.blocks.0.attn.qkv.weight', 'layers.0.blocks.0.attn.qkv.bias', 'layers.0.blocks.0.attn.proj.weight', 'layers.0.blocks.0.attn.proj.bias', 'layers.0.blocks.0.norm2.weight', 'layers.0.blocks.0.norm2.bias', 'layers.0.blocks.0.mlp.fc1.weight', 'layers.0.blocks.0.mlp.fc1.bias', 'layers.0.blocks.0.mlp.fc2.weight', 'layers.0.blocks.0.mlp.fc2.bias', 'layers.0.blocks.1.norm1.weight', 'layers.0.blocks.1.norm1.bias', 'layers.0.blocks.1.attn.relative_position_bias_table', 'layers.0.blocks.1.attn.relative_position_index', 'layers.0.blocks.1.attn.qkv.weight', 'layers.0.blo

In [9]:



# model.eval()
# #只保留权重
# torch.save(model.state_dict(), 'model_no_arc.pth')


checkpoint = torch.load('F:/gta5/swin-large-p244-w877_in22k-pre_16xb8-amp-32x2x1-30e_kinetics700-rgb_20220930-f8d74db7.pth')

checkpoint2=torch.load('F:/A-ROAD-CHALL/weights/weights_task1/pretrained_weights_task1.pth')
# # # # model.load_state_dict(checkpoint, strict=False)

# # # # load_checkpoint(model, checkpoint, strict=False,revise_keys=[r'^backbone\.'])

for k in list(checkpoint.keys()):
    # 去掉backbone.
    new_k=k
    if k.startswith('backbone'):
        new_k = k[9:] 
    # 修改层名    
    new_k = new_k.replace('layers.0', 'swinaclayer1')
    new_k = new_k.replace('layers.1', 'swinaclayer2')
    new_k = new_k.replace('layers.2', 'swinaclayer3')
    # print('ok')
    new_k = new_k.replace('layers.3', 'swinaclayer4')
    # 改键名
    checkpoint[new_k] = checkpoint.pop(k)



for k in list(checkpoint2.keys()):
    new_k=k
    if k.startswith('module.backbone'):
        new_k = k[16:]
        # print('ok')
    #改键名
    checkpoint2[new_k] = checkpoint2.pop(k)


# new_checkpoint = {}
# new_checkpoint.update(checkpoint) 
# new_checkpoint.update(checkpoint2)

# 加载到模型中    
# model.load_state_dict(new_checkpoint, strict=False)
# model.load_state_dict(checkpoint2, strict=False)
model.load_state_dict(checkpoint, strict=False)


# summary(model, input_size=(1, 3, 8, 224, 224))
# print(model)


_IncompatibleKeys(missing_keys=['patch_embed.proj.weight', 'patch_embed.proj.bias', 'patch_embed.norm.weight', 'patch_embed.norm.bias', 'swinaclayer1.blocks.0.norm1.weight', 'swinaclayer1.blocks.0.norm1.bias', 'swinaclayer1.blocks.0.attn.relative_position_bias_table', 'swinaclayer1.blocks.0.attn.relative_position_index', 'swinaclayer1.blocks.0.attn.qkv.weight', 'swinaclayer1.blocks.0.attn.qkv.bias', 'swinaclayer1.blocks.0.attn.proj.weight', 'swinaclayer1.blocks.0.attn.proj.bias', 'swinaclayer1.blocks.0.norm2.weight', 'swinaclayer1.blocks.0.norm2.bias', 'swinaclayer1.blocks.0.mlp.fc1.weight', 'swinaclayer1.blocks.0.mlp.fc1.bias', 'swinaclayer1.blocks.0.mlp.fc2.weight', 'swinaclayer1.blocks.0.mlp.fc2.bias', 'swinaclayer1.blocks.1.norm1.weight', 'swinaclayer1.blocks.1.norm1.bias', 'swinaclayer1.blocks.1.attn.relative_position_bias_table', 'swinaclayer1.blocks.1.attn.relative_position_index', 'swinaclayer1.blocks.1.attn.qkv.weight', 'swinaclayer1.blocks.1.attn.qkv.bias', 'swinaclayer1.bloc

In [10]:
#good

""" 

FPN network Classes

Author: Gurkirt Singh
Inspired from https://github.com/kuangliu/pytorch-retinanet and
https://github.com/gurkirt/realtime-action-detection

"""

import math

import torch
import torch.nn as nn

import modules.utils as utils
# from backbone_models import backbone_models
from modules.anchor_box_kmeans import anchorBox as KanchorBox
from modules.anchor_box_retinanet import anchorBox as RanchorBox
from modules.box_utils import decode
from modules.detection_loss import FocalLoss

logger = utils.get_logger(__name__)

class RetinaNet(nn.Module):
    """Feature Pyramid Network Architecture
    The network is composed of a backbone FPN network followed by the
    added Head conv layers.  
    Each head layer branches into
        1) conv2d for class conf scores
        2) conv2d for localization predictions
    See: 
    RetinaNet: https://arxiv.org/pdf/1708.02002.pdf for more details.
    FPN: https://arxiv.org/pdf/1612.03144.pdf

    Args:
        backbone Network:
        Program Argument Namespace

    """

    def __init__(self, backbone, args):
        super(RetinaNet, self).__init__()

        self.num_classes = args.num_classes
        # TODO: implement __call__ i

        if args.ANCHOR_TYPE == 'RETINA':
            self.anchors = RanchorBox()
        elif args.ANCHOR_TYPE == 'KMEANS':
            self.anchors = KanchorBox()
        else:
            raise RuntimeError('Define correct anchor type')
            
        # print('Cell anchors\n', self.anchors.cell_anchors)
        # pdb.set_trace()
        self.ar = self.anchors.ar
        args.ar = self.ar
        self.use_bias = True
        self.head_size = 256
        self.backbone = backbone
        self.SEQ_LEN = args.SEQ_LEN
        self.HEAD_LAYERS = args.HEAD_LAYERS
        self.NUM_FEATURE_MAPS = args.NUM_FEATURE_MAPS
        
        self.reg_heads = []
        self.cls_heads = []
        self.prior_prob = 0.01
        bias_value = -math.log((1 - self.prior_prob) / self.prior_prob)
        # for nf in range(self.NUM_FEATURE_MAPS):
        self.reg_heads = self.make_head(
            self.ar * 4, args.REG_HEAD_TIME_SIZE, self.HEAD_LAYERS)
        self.cls_heads = self.make_head(
            self.ar * self.num_classes, args.CLS_HEAD_TIME_SIZE, self.HEAD_LAYERS)
        
        nn.init.constant_(self.cls_heads[-1].bias, bias_value)

        if args.MODE == 'train':  # eval_iters only in test case
            self.criterion = FocalLoss(args)

        # self.ego_head = nn.Conv3d(self.head_size, args.num_ego_classes, kernel_size=(
        #     3, 1, 1), stride=1, padding=(1, 0, 0))
        # nn.init.constant_(self.ego_head.bias, bias_value)


    # def forward(self, images, gt_boxes=None, gt_labels=None, ego_labels=None, counts=None, img_indexs=None, get_features=False):
    def forward(self, images, gt_boxes=None, gt_labels=None, counts=None, img_indexs=None, get_features=False, logic=None, Cplus=None, Cminus=None, is_ulb=None):
        # sources, ego_feat = self.backbone(images)
        sources = self.backbone(images)

        # ego_preds = self.ego_head(
        #     ego_feat).squeeze(-1).squeeze(-1).permute(0, 2, 1).contiguous()

        grid_sizes = [feature_map.shape[-2:] for feature_map in sources]
        ancohor_boxes = self.anchors(grid_sizes)
        
        loc = list()
        conf = list()

        for x in sources:
            loc.append(self.reg_heads(x).permute(0, 2, 3, 4, 1).contiguous())
            conf.append(self.cls_heads(x).permute(0, 2, 3, 4, 1).contiguous())

        loc = torch.cat([o.view(o.size(0), o.size(1), -1) for o in loc], 2)
        conf = torch.cat([o.view(o.size(0), o.size(1), -1) for o in conf], 2)

        flat_loc = loc.view(loc.size(0), loc.size(1), -1, 4)
        flat_conf = conf.view(conf.size(0), conf.size(1), -1, self.num_classes)

        # pdb.set_trace()
        if get_features:  # testing mode with feature return
            return flat_conf, features
        elif gt_boxes is not None:  # training mode
            # return self.criterion(flat_conf, flat_loc, gt_boxes, gt_labels, counts, ancohor_boxes, ego_preds, ego_labels)
            # return self.criterion(flat_conf, flat_loc, gt_boxes, gt_labels, counts, ancohor_boxes)
            return self.criterion(flat_conf, flat_loc, gt_boxes, gt_labels, counts, ancohor_boxes, logic, Cplus, Cminus, is_ulb=is_ulb), is_ulb

        else:  # otherwise testing mode
            decoded_boxes = []
            for b in range(flat_loc.shape[0]):
                temp_l = []
                for s in range(flat_loc.shape[1]):
                    # torch.stack([decode(flat_loc[b], ancohor_boxes) for b in range(flat_loc.shape[0])], 0),
                    temp_l.append(decode(flat_loc[b, s], ancohor_boxes))
                decoded_boxes.append(torch.stack(temp_l, 0))
            # return torch.stack(decoded_boxes, 0), flat_conf, ego_preds
            return torch.stack(decoded_boxes, 0), flat_conf


    def make_features(self,  shared_heads):
        layers = []
        use_bias = self.use_bias
        head_size = self.head_size
        for _ in range(shared_heads):
            layers.append(nn.Conv3d(head_size, head_size, kernel_size=(
                1, 3, 3), stride=1, padding=(0, 1, 1), bias=use_bias))
            layers.append(nn.ReLU(True))

        layers = nn.Sequential(*layers)

        for m in layers.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.normal_(m.weight, mean=0, std=0.01)
                if hasattr(m.bias, 'data'):
                    nn.init.constant_(m.bias, 0)

        return layers

    def make_head(self, out_planes, time_kernel,  num_heads_layers):
        layers = []
        use_bias = self.use_bias
        head_size = self.head_size
        
        for kk in range(num_heads_layers):
            # if kk % 2 == 1 and time_kernel>1:
            #     branch_kernel = 3
            #     bpad = 1
            # else:
            branch_kernel = 1
            bpad = 0
            layers.append(nn.Conv3d(head_size, head_size, kernel_size=(
                branch_kernel, 3, 3), stride=1, padding=(bpad, 1, 1), bias=use_bias))
            layers.append(nn.ReLU(True))

        tpad = time_kernel//2
        layers.append(nn.Conv3d(head_size, out_planes, kernel_size=(
            time_kernel, 3, 3), stride=1, padding=(tpad, 1, 1)))
        
        layers = nn.Sequential(*layers)

        for m in layers.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.normal_(m.weight, mean=0, std=0.01)
                if hasattr(m.bias, 'data'):
                    nn.init.constant_(m.bias, 0)

        return layers


# def build_retinanet(args):
#     return RetinaNet(backbone_models(args), args)


In [12]:



modelact=RetinaNet(model, args)

summary(model, input_size=(1, 3, 8, 224, 224),depth=150000)

Layer (type:depth-idx)                        Output Shape              Param #
ResNetFPN                                     [1, 256, 4, 28, 28]       28,100,928
├─PatchEmbed3D: 1-1                           [1, 192, 4, 56, 56]       --
│    └─Conv3d: 2-1                            [1, 192, 4, 56, 56]       18,624
│    └─LayerNorm: 2-2                         [1, 12544, 192]           384
├─Dropout: 1-2                                [1, 192, 4, 56, 56]       --
├─BasicLayer: 1-3                             [1, 384, 4, 28, 28]       --
│    └─ModuleList: 2-3                        --                        --
│    │    └─SwinTransformerBlock3D: 3-1       [1, 4, 56, 56, 192]       --
│    │    │    └─LayerNorm: 4-1               [1, 4, 56, 56, 192]       384
│    │    │    └─WindowAttention3D: 4-2       [64, 196, 192]            15,210
│    │    │    │    └─Linear: 5-1             [64, 196, 576]            111,168
│    │    │    │    └─Softmax: 5-2            [64, 6, 196, 196]         

In [5]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='Training single stage FPN with OHEM, resnet as backbone')
    args, unknown = parser.parse_known_args()
    args.model_perms = [3, 4, 6, 3]
    args.model_3d_layers = [[0, 1, 2], [0, 2], [0, 2, 4], [0, 1]]
    args.non_local_inds=[[], [], [], []]
    main()

d:\Users\belongTH\anaconda3\envs\testpyt\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


ResNetFPN(
  (patch_embed): PatchEmbed3D(
    (proj): Conv3d(3, 192, kernel_size=(2, 4, 4), stride=(2, 4, 4))
    (norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (swinaclayer1): BasicLayer(
    (blocks): ModuleList(
      (0): SwinTransformerBlock3D(
        (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (attn): WindowAttention3D(
          (qkv): Linear(in_features=192, out_features=576, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=192, out_features=192, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
          (softmax): Softmax(dim=-1)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=192, out_features=768, bias=True)
          (act): GELU(approximate=none)
          (fc2): Linear(in_features=768, out_feature

In [40]:
# 加载Swin Transformer的权重
import torch
from torchvision.models import resnet



AttributeError: 'Args' object has no attribute 'MODEL_TYPE'

In [ ]:
# 加载neck和head的权重
neck_head_weights = torch.load('F:/A-ROAD-CHALL/weights/weights_task1/pretrained_weights_task1.pth')

# 只取neck和head部分的权重
neck_head_weights = {k: v for k, v in neck_head_weights.items() if k.startswith('layer1') or k.startswith('corr') or k.startswith('conv6') or k.startswith('conv7')}

# 加载到模型中
model.load_state_dict(neck_head_weights, strict=False)

In [4]:
for i in range(1, 5):
    layer_name = 'swinaclayer{}'.format(i)
    print(i)
    print(layer_name)


    

1
swinaclayer1
2
swinaclayer2
3
swinaclayer3
4
swinaclayer4
